In [1]:
import math
%matplotlib inline
# from tomkin import detect_rpeak
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
# from outlier_calculation import Quality,compute_outlier_ecg
# from hrvanalysis import remove_ectopic_beats
from joblib import Parallel,delayed
# from data_quality import compute_quality,ECGQualityCalculation_BLE,ECGQualityCalculation
from joblib import delayed,Parallel
# from decode import Preprc
from scipy import signal
import shutil
import math
from sklearn.preprocessing import RobustScaler
import pickle
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as pat
from sklearn.preprocessing import RobustScaler
from scipy import signal
%matplotlib inline
from scipy.stats import skew,kurtosis
import numpy as np
from scipy import signal
from scipy.stats import iqr
import pickle
import warnings
warnings.filterwarnings('ignore')
def complementary_filter(accel_gyro, fq=25):
    ts = accel_gyro[:,0]
    acc_x = accel_gyro[:,5]
    acc_y = accel_gyro[:,6]
    acc_z = accel_gyro[:,7]
    gyr_x = accel_gyro[:,8]
    gyr_y = accel_gyro[:,9]
    gyr_z = accel_gyro[:,10]

    dt = 1.0 / fq  # 1/16.0;
    M_PI = math.pi
    hpf = 0.85
    lpf = 0.15

    thetaX_acc = [0] * len(acc_x)  # math.atan2(-acc_z,acc_y)*180/M_PI;
    thetaY_acc = [0] * len(acc_x)  # math.atan2(acc_x,acc_z)*180/M_PI;
    thetaZ_acc = [0] * len(acc_x)  # math.atan2(acc_y,acc_x)*180/M_PI;

    thetaX = [0] * len(gyr_x)
    thetaY = [0] * len(gyr_y)
    thetaZ = [0] * len(gyr_z)

    for index in range(len(gyr_x)):
        thetaX_acc[index] = math.atan2(-acc_z[index], acc_y[index]) * 180 / M_PI
        thetaY_acc[index] = math.atan2(acc_x[index], acc_z[index]) * 180 / M_PI
        thetaZ_acc[index] = math.atan2(acc_y[index], acc_x[index]) * 180 / M_PI

        if index == 0:
            thetaX[index] = hpf * thetaX[index] * dt + lpf * thetaX_acc[index]
            thetaY[index] = hpf * thetaY[index] * dt + lpf * thetaY_acc[index]
            thetaZ[index] = hpf * thetaZ[index] * dt + lpf * thetaZ_acc[index]
        else:
            thetaX[index] = hpf * (thetaX[index - 1] + gyr_x[index] * dt) + lpf * thetaX_acc[index]
            thetaY[index] = hpf * (thetaY[index - 1] + gyr_y[index] * dt) + lpf * thetaY_acc[index]
            thetaZ[index] = hpf * (thetaZ[index - 1] + gyr_z[index] * dt) + lpf * thetaZ_acc[index]

    roll_pitch_yaw = np.zeros((len(ts),7))
    roll_pitch_yaw[:,0] = ts
    roll_pitch_yaw[:,1:4] =  accel_gyro[:,1:4]
    roll_pitch_yaw[:,4] = thetaX
    roll_pitch_yaw[:,5] = thetaY
    roll_pitch_yaw[:,6] = thetaZ
    return roll_pitch_yaw

def get_metric(window):
    f,pxx = signal.welch(window[:,1],fs=25,nperseg=len(window[:,1]),nfft=1000)
    pxx = np.abs(pxx)
    pxx = pxx/max(pxx)
    return np.array([skew(window[:,1]),kurtosis(window[:,1]),np.trapz(pxx[np.where((f>=.8)&(f<=2.5))[0]])/np.trapz(pxx),
                len(np.where(np.diff(np.signbit(window[:,1])))[0])/len(window[:,1])]).reshape(-1,4)

def get_classified_data(ppg_data,final_path,r,clf,ecg_rr):
    print(ppg_data.shape)
    roll_pitch_yaw = complementary_filter(ppg_data)
    print(roll_pitch_yaw.shape)
    final_ppg_data = np.zeros((ppg_data.shape[0],20))
    final_ppg_data[:,14:17] = roll_pitch_yaw[:,4:]
    print(roll_pitch_yaw.shape,'again')
    index_col = []
    chnls = [np.zeros((0,4)),np.zeros((0,4)),np.zeros((0,4))]
    for i in range(30,len(ppg_data[:,0])-30,1):
        t = ppg_data[i,0]
        final_ppg_data[i,:11] = ppg_data[i,:]
        ppg_window = ppg_data[i-30:i+30,:]
        temp_window = RobustScaler().fit_transform(ppg_window[:,2:5])
        final_ppg_data[i,17:] = temp_window[ppg_window[:,0]==t,:]
        index_col.append(i)
        for k in range(2,5,1):
            window = np.concatenate([ppg_window[:,0].reshape(-1,1),signal.detrend(temp_window[:,k-2]).reshape(-1,1)],
                                    axis=1)
            f = get_metric(window)
            chnls[k-2] = np.concatenate((chnls[k-2],f))
        if i%1000==0:
            print(i)
    index_col = np.array(index_col)
    for i in range(2,5,1):
        final_ppg_data[index_col,11+k-2] = clf.predict_proba(chnls[k-2])[:,1]
    final_ppg_data = final_ppg_data[final_ppg_data[:,0]>0,:]
    print(final_ppg_data.shape,'final')
    return final_ppg_data,final_path,r,ecg_rr

In [2]:
final_data = pickle.load(open('./data_saved/temp_data_ecg.p','rb'))

In [ ]:
import numpy as np
final_data1 = []
for a in final_data:
    if len(a[0])<10000:
        continue
    a[0][:,5:8] = a[0][:,5:8]*2/16384
    a[0][:,8:] = 500.0 * a[0][:,8:] / 32768
    split_data = np.array_split(a[0],10)
    for b in split_data:
        final_data1.append([b,a[1],a[2]])
print(len(final_data))
final_data = final_data1
print(len(final_data1))

In [ ]:
clf = pickle.load(open('./temp/classifier.p','rb'))
from joblib import Parallel,delayed
# final_data2 = [get_classified_data(a[0],a[1],a[2],clf,a[3]) for a in final_data]
final_data2 = Parallel(n_jobs=20,verbose=2)(delayed(get_classified_data)(a[0],a[1],a[2],clf,a[3]) for a in final_data)
pickle.dump(final_data2,open('./data_saved/data_all_jhu_processed.p','wb'))

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed: 112.5min


In [ ]:
import pickle
import numpy as np
final_data = pickle.load(open('./data_saved/data_all_mperf_processed.p','rb'))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(final_data[0][0][7000:8000,14:17])
plt.show()
plt.plot(final_data[0][0][7000:8000,17:])
plt.show()
plt.plot(final_data[0][0][7000:8000,11:14])
plt.show()

In [ ]:
# pickle.dump([window_col,label_col],open('./data_saved/windows_all_inertail.p','wb'))

In [ ]:
pickle.dump(final_data,open('./data_saved/data_all_mperf.p','wb'))

In [ ]:
label_col = np.array([np.array(a) for a in label_col])

In [ ]:
x = [np.var(np.sum(np.abs(a[:,5:8]),axis=1)) for a in window_col]

In [ ]:
feature = np.zeros((label_col.shape[0],2))

In [ ]:
feature[:,1] = x
feature[:,0] = label_col[:,-1]

In [ ]:
plt.scatter(x,np.max(label_col,axis=1))
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
df = pd.DataFrame(feature,columns=['x','y'])
sns.jointplot(x='x', y='y', kind="kde", color="k",data=df)
plt.ylim([0,.25])

In [ ]:
len(x)